In [1]:
#pip install snowflake-connector-python sqlalchemy pandas matplotlib ta yfinance pymysql snowflake-sqlalchemy python-dotenv cryptography simplejson pandas_datareader

In [2]:
#pip install pyqt5  plotly

In [3]:
#pip install --upgrade -r requirements.txt

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from datetime import datetime,timedelta
import requests
import ta

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import base64
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from sqlalchemy import text

load_dotenv()

with open(os.getenv("PRIVATE_KEY_PATH"), "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )

private_key_pkcs8 = base64.b64encode(
    private_key.private_bytes(
        encoding=serialization.Encoding.DER,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
).decode("utf-8")  # ✅ base64 string, not bytes!

engine = create_engine(URL(
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    user=os.getenv("SNOWFLAKE_USER"),
    private_key=private_key_pkcs8,
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
    role=os.getenv("SNOWFLAKE_ROLE")
))

with engine.connect() as conn:
    result = conn.execute(text("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_TIMESTAMP();"))
    for row in result:
        print("✅ Snowflake connected:", row)


✅ Snowflake connected: ('CRYPTO_USER', 'CRYPTO_ROLE', datetime.datetime(2025, 4, 9, 8, 56, 32, 11000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>))


In [4]:
from datetime import datetime

def convert_to_date(date_string, date_format="%Y-%m-%d"):
    """
    Converts a date string to a datetime object.

    Args:
        date_string (str): The date as a string.
        date_format (str): The format of the date string (default is '%Y-%m-%d').

    Returns:
        datetime: A datetime object representing the date, or None if conversion fails.
    """
    try:
        return datetime.strptime(date_string, date_format)
    except ValueError:
        print(f"Error: '{date_string}' does not match the format '{date_format}'")
        return None



In [5]:
import requests
import time
import pandas as pd

def fetch_bitstamp_history(start_date='2011-09-01', end_date='2014-09-14'):
    print("📥 Fetching BTC/USD history from Bitstamp...")

    url = 'https://www.bitstamp.net/api/v2/ohlc/btcusd/'
    all_data = []

    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    while start <= end:
        params = {
            'step': 86400,  # daily
            'limit': 1000,
            'start': int(start.timestamp()),
            'end': int((start + pd.Timedelta(days=999)).timestamp())
        }

        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"❌ Failed at {start.date()}")
            break

        data = response.json()['data']['ohlc']
        all_data.extend(data)

        start += pd.Timedelta(days=1000)
        time.sleep(1)  # avoid rate limits

    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.rename(columns={
        'timestamp': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'volume': 'Volume'
    })
    df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
    df = df.sort_values("Date").reset_index(drop=True)

    print(f"✅ Retrieved {len(df)} rows from Bitstamp.")

    # Add missing days (2014-09-15 and 2014-09-16) by forward-filling
    last_row = df[df['Date'] == pd.to_datetime('2014-09-14')]
    if not last_row.empty:
        fill_dates = [pd.to_datetime('2014-09-15'), pd.to_datetime('2014-09-16')]
        filled_rows = pd.DataFrame([{
            'Date': d,
            'Open': last_row['Open'].values[0],
            'High': last_row['High'].values[0],
            'Low': last_row['Low'].values[0],
            'Close': last_row['Close'].values[0],
            'Volume': 0.0  # or same volume if you prefer
        } for d in fill_dates])
        df = pd.concat([df, filled_rows]).sort_values('Date').reset_index(drop=True)
        print(f"🧩 Forward-filled missing dates: {[d.date() for d in fill_dates]}")

    return df



In [6]:
import yfinance as yf


def fetch_index_history(ticker, start='2011-01-01'):
    print(f"📥 Fetching {ticker} history from Yahoo Finance...")

    df = yf.download(ticker, start=start, progress=False)

    if df.empty:
        raise ValueError(f"❌ No data returned for {ticker}. Check the symbol or your connection.")

    # Flatten MultiIndex columns if present
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [col[0] for col in df.columns]

    df.reset_index(inplace=True)

    # Rename columns for consistency
    df = df.rename(columns={'Adj Close': 'Adj_Close'})

    # Define standard columns
    base_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    if 'Adj_Close' in df.columns:
        base_cols.append('Adj_Close')  # only include if it exists

    df = df[[col for col in base_cols if col in df.columns]]

    print(f"✅ Retrieved {len(df)} rows for {ticker}")
    return df





In [7]:
bitcoin_full_load = False
btc_df = []

if bitcoin_full_load:
  # Fetch historical BTC price data
  bitstamp_df = fetch_bitstamp_history()
  btc_df = fetch_index_history('BTC-USD')
  # Merge datasets
  btc_df = pd.concat([bitstamp_df, btc_df])
  btc_df = btc_df.sort_values('Date').reset_index(drop=True)
  print(f"📊 Final dataset has {len(btc_df)} rows, from {btc_df['Date'].min().date()} to {btc_df['Date'].max().date()}")
else:
  btc_df = fetch_index_history('BTC-USD')

btc_df.head()


📥 Fetching BTC-USD history from Yahoo Finance...
YF.download() has changed argument auto_adjust default to True
✅ Retrieved 3858 rows for BTC-USD


,Date,Open,High,Low,Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100


In [10]:
from sqlalchemy import text
from sqlalchemy.types import Float, DateTime

def upsert_timeseries_to_snowflake(df, table_name, engine, schema='PUBLIC', database='CRYPTODB'):
    """
    Appends new rows to a Snowflake table if they don't already exist (based on Date column).
    Creates the table if it does not exist.

    Parameters:
        df (pd.DataFrame): DataFrame with a 'Date' column.
        table_name (str): Target table name in Snowflake.
        engine: SQLAlchemy engine for Snowflake.
        schema (str): Schema name (default is 'PUBLIC').
        database (str): Database name (default is 'CRYPTODB').
    """

    with engine.connect() as conn:
        existing_dates = set()

        # Step 1: Check if table exists
        result = conn.execute(text(f"""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = '{schema}' AND table_name = '{table_name.upper()}'
        """)).fetchone()

        if result is not None:
            print(f"📌 Table {table_name.upper()} exists. Checking for existing data...")
            query = f'SELECT DISTINCT "Date" FROM {database}.{schema}."{table_name.upper()}"'
            date_rows = conn.execute(text(query)).fetchall()
            existing_dates = {row[0].date() for row in date_rows}
        else:
            print(f"📌 Table {table_name.upper()} does not exist. It will be created.")

    # Step 2: Filter only new rows
    df['Date_only'] = df['Date'].dt.date
    new_data = df[~df['Date_only'].isin(existing_dates)].drop(columns='Date_only')

    # Step 3: Insert new rows if needed
    if not new_data.empty:
        print(f"🆕 Appending {len(new_data)} new rows to {table_name.upper()}...")
        new_data.to_sql(
            name=table_name,
            con=engine,
            schema=schema,
            if_exists='append',
            index=False,
            dtype={
                "Date": DateTime(),
                "Open": Float(),
                "High": Float(),
                "Low": Float(),
                "Close": Float(),
                "Volume": Float()
            }
        )
    else:
        print(f"✅ No new data to append — Snowflake table {table_name.upper()} is up-to-date.")



In [11]:
from sqlalchemy.types import Integer, String, Float, DateTime
nasdaq_df = fetch_index_history('^IXIC')
sp500_df = fetch_index_history('^GSPC')
vix_df = fetch_index_history('^VIX')

📥 Fetching ^IXIC history from Yahoo Finance...
✅ Retrieved 3589 rows for ^IXIC
📥 Fetching ^GSPC history from Yahoo Finance...
✅ Retrieved 3589 rows for ^GSPC
📥 Fetching ^VIX history from Yahoo Finance...
✅ Retrieved 3589 rows for ^VIX


In [13]:
#with engine.connect() as conn:
#    conn.execute(text('DROP TABLE IF EXISTS nasdaq'))
#   conn.execute(text('DROP TABLE IF EXISTS sp500'))
#    conn.execute(text('DROP TABLE IF EXISTS vix'))

In [12]:
upsert_timeseries_to_snowflake(btc_df, 'btcusd', engine)
upsert_timeseries_to_snowflake(nasdaq_df, 'nasdaq', engine)
upsert_timeseries_to_snowflake(sp500_df, 'sp500', engine)
upsert_timeseries_to_snowflake(vix_df, 'vix', engine)

📌 Table BTCUSD exists. Checking for existing data...
✅ No new data to append — Snowflake table BTCUSD is up-to-date.
📌 Table NASDAQ exists. Checking for existing data...
🆕 Appending 1 new rows to NASDAQ...
📌 Table SP500 exists. Checking for existing data...
🆕 Appending 1 new rows to SP500...
📌 Table VIX exists. Checking for existing data...
🆕 Appending 1 new rows to VIX...


In [13]:
from sqlalchemy import text
from sqlalchemy.types import Integer, String, Float, DateTime

# Fetch data from Fear & Greed Index API
url = 'https://api.alternative.me/fng/?limit=0'
response = requests.get(url)
response.raise_for_status()
data = response.json()['data']

# Convert to DataFrame
df = pd.DataFrame(data)

# Convert 'timestamp' to UTC and then remove timezone info
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)
df['timestamp'] = df['timestamp'].dt.tz_localize(None)



df['value'] = pd.to_numeric(df['value'])


# Drop unneeded columns
df.drop(columns=['time_until_update','value_classification'], inplace=True)

with engine.connect() as conn:
    conn.execute(text('DROP TABLE IF EXISTS feargreedindex'))






C:\Users\edufs\AppData\Local\Temp\ipykernel_21944\3101141023.py:14: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)


In [14]:
df.to_sql(
    'feargreedindex',
    con=engine,
    if_exists='replace',
    index=False,
    dtype={
        "value": Float(),
        "timestamp": DateTime()
    }

)

print("✅ FearGreedIndex table successfully written to Snowflake.")

✅ FearGreedIndex table successfully written to Snowflake.


In [15]:
query = """
SELECT  BTC.* , cast(fgi.value AS Integer) as Fgi,
NASD."Close" AS NASDAQ, SP500."Close" SP500, VIX."Close" as VIX
FROM CRYPTODB.PUBLIC.BTCUSD btc
LEFT JOIN CRYPTODB.PUBLIC.FEARGREEDINDEX fgi
    ON btc."Date" = fgi.timestamp
LEFT JOIN CRYPTODB.PUBLIC.NASDAQ NASD
    ON BTC."Date" = NASD."Date"
LEFT JOIN CRYPTODB.PUBLIC.SP500 SP500
    ON BTC."Date" = SP500."Date"
LEFT JOIN CRYPTODB.PUBLIC.VIX VIX
    ON BTC."Date" = VIX."Date"
Order by 1
"""

data = pd.read_sql(query,con=engine)
data.head()

,Date,Open,High,Low,Close,Volume,fgi,nasdaq,sp500,vix
0,2011-09-01,8.00,8.31,5.25,8.22,164.480000,NaN,2546.040039,1204.420044,31.820000
1,2011-09-02,8.12,8.88,8.00,8.88,79.760000,NaN,2480.330078,1173.969971,33.919998
2,2011-09-03,8.88,8.88,8.88,8.88,0.000000,NaN,NaN,NaN,NaN
3,2011-09-04,8.89,8.89,8.89,8.89,0.200225,NaN,NaN,NaN,NaN
4,2011-09-05,8.89,8.89,8.89,8.89,0.000000,NaN,NaN,NaN,NaN


In [16]:
# Step 1: Create BTC return and volatility features
data['btc_7d_return'] = data['Close'].pct_change(7)
data['btc_volatility'] = data['Close'].rolling(window=14).std()

# Step 2: Build FGI proxy based on returns and volatility
fgi_proxy = 100 * (data['btc_7d_return'].rank(pct=True) - data['btc_volatility'].rank(pct=True))
fgi_proxy = fgi_proxy.clip(lower=0, upper=100)

# Step 3: Use real FGI where available, otherwise fill with proxy
data['Fgi_Full'] = np.where(
    data['Date'] < pd.to_datetime('2018-02-01'),
    fgi_proxy,
    data['fgi']
)




In [17]:
data['Fgi_Full'].isna().sum()

np.int64(17)

In [18]:
# Columns to forward fill only
macro_cols = [ 'nasdaq','sp500','vix','Fgi_Full']

# Apply forward fill just to those columns
data[macro_cols] = data[macro_cols].ffill()

# Calculate the average of the FGI column, excluding NaN values
fg_average = data['Fgi_Full'].mean()

# Fill NaN values in the FGI column with the calculated average
data['Fgi_Full'].fillna(fg_average, inplace=True)

data['Fgi_Full'] = data['Fgi_Full'].round().astype(int)

# Get the unique values of 'B' column
#unique_values = data['Fgi_Full'].unique()

#print (unique_values)

C:\Users\edufs\AppData\Local\Temp\ipykernel_21944\4251775203.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Fgi_Full'].fillna(fg_average, inplace=True)


In [19]:
import plotly.express as px

fig = px.box(
    data_frame=data,
    y='Close',
    points='all',  # show all points; use 'outliers' or False if preferred
    title='📦 Boxplot of Bitcoin Closing Prices',
    labels={'Close': 'BTC Close Price (USD)'},
)

fig.update_layout(
    yaxis_title='BTC Close Price (USD)',
    template='plotly_white'
)

fig.show()



In [20]:
import plotly.express as px

fig = px.histogram(
    data_frame=data,
    x='Close',
    nbins=50,
    title='📈 Distribution of Bitcoin Closing Prices',
    labels={'Close': 'BTC Close Price (USD)'},
    opacity=0.75
)

fig.update_layout(
    xaxis_title='BTC Close Price (USD)',
    yaxis_title='Frequency',
    bargap=0.1,
    template='plotly_white'
)

fig.show()


In [21]:
data['Year'] = data['Date'].dt.year

fig = px.box(
    data,
    x='Year',
    y='Close',
    points='outliers',
    title='📦 BTC Price Distribution by Year',
    labels={'Close': 'BTC Close Price (USD)', 'Year': 'Year'},
)

fig.update_layout(template='plotly_white')
fig.show()


In [22]:
data['FGI_Level'] = pd.cut(data['Fgi_Full'], bins=[0, 25, 50, 75, 100],
                           labels=['Extreme Fear', 'Fear', 'Greed', 'Extreme Greed'])

fig = px.box(
    data_frame=data,
    x='FGI_Level',
    y='Close',
    points='outliers',
    title='📦 BTC Price Distribution by Fear & Greed Level',
    labels={'Close': 'BTC Close Price (USD)', 'FGI_Level': 'Fear & Greed Index'},
    template='plotly_white'
)

fig.show()


In [25]:
print(data[:].isna().sum())

Date                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
fgi               2657
nasdaq               0
sp500                0
vix                  0
btc_7d_return        7
btc_volatility      13
Fgi_Full             0
Year                 0
FGI_Level          631
dtype: int64


In [23]:
#https://www.investopedia.com/terms/m/macd.asp
#macd_object = ta.trend.MACD(data['close'])
macd_object = ta.trend.MACD(data['Close'], window_slow = 26, window_fast = 12, window_sign = 9, fillna= True)
data['MACD'] = macd_object.macd()
data['MACD_Signal'] = macd_object.macd_signal()
data['MACD_Diff'] = macd_object.macd_diff()
#data.tail()

In [24]:
#https://www.investopedia.com/terms/m/mfi.asp
mfi_indicator = ta.volume.MFIIndicator(high=data['High'], low=data['Low'], close=data['Close'], volume=data['Volume'], window=14,fillna=True)
data['mfi'] = mfi_indicator.money_flow_index()

In [25]:
# RSI
rsi = ta.momentum.rsi(data['Close'], window=14, fillna=True)
data['RSI'] = rsi

In [26]:
# EMA
ema_short = ta.trend.EMAIndicator(data['Close'], window=12, fillna=True)
ema_long = ta.trend.EMAIndicator(data['Close'], window=26, fillna=True)
data['EMA_Short'] = ema_short.ema_indicator()
data['EMA_Long'] = ema_long.ema_indicator()

In [27]:
# Bollinger Bands
bollinger = ta.volatility.BollingerBands(data['Close'], window=20, window_dev=2, fillna=True)
data['Bollinger_Upper'] = bollinger.bollinger_hband()
data['Bollinger_Lower'] = bollinger.bollinger_lband()
data['Bollinger_Middle'] = bollinger.bollinger_mavg()

In [28]:
# Stochastic Oscillator
stoch = ta.momentum.StochasticOscillator(data['High'], data['Low'], data['Close'], window=14, smooth_window=3, fillna=True)
data['Stochastic'] = stoch.stoch()
data['Stochastic_Signal'] = stoch.stoch_signal()

In [29]:
# ATR
atr = ta.volatility.AverageTrueRange(data['High'], data['Low'], data['Close'], window=14, fillna=True)
data['ATR'] = atr.average_true_range()

In [30]:
# Parabolic SAR
sar = ta.trend.PSARIndicator(data['High'], data['Low'], data['Close'], fillna=True)
data['SAR'] = sar.psar()

In [31]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare the data (assuming 'data' is already pre-processed with features)
data['Date'] = pd.to_datetime(data['Date'], utc=True)
data['Target'] = data['Close'].shift(-1)
data['Return'] = data['Close'].pct_change().fillna(0)
data['Volatility'] = data['Return'].rolling(window=7).std().fillna(0)
data['Volatility_x_Return'] = data['Volatility'] * data['Return']

data = data.dropna(subset=['Target'])

# Select features (excluding 'Date')
features = [
     'Bollinger_Upper', 'Bollinger_Lower', 'Fgi_Full', 'Close','nasdaq','sp500','vix','Volume'
]

X = data[features]
y = data['Target']

In [32]:
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [33]:
# Standardize the features
scaler = StandardScaler()
scaler.fit(X_train)

# Standardize the features
# Formula: z = (x - μ) / σ
# Where: x = original value, μ = mean of feature, σ = standard deviation of feature

# Then scale:
x_train_scaled = scaler.transform(X_train)
x_test_scaled = scaler.transform(X_test)


# Train the model
model = LinearRegression()
model.fit(x_train_scaled, y_train)

# Make predictions on the test data
y_pred = model.predict(x_test_scaled)

In [34]:
x_train_scaled_df = pd.DataFrame(x_train_scaled, columns=X_train.columns)

# Print min and max per feature
print("Min and Max of standardized features (X_train):")
for col in x_train_scaled_df.columns:
    col_min = np.min(x_train_scaled_df[col])
    col_max = np.max(x_train_scaled_df[col])
    print(f"{col}: min = {col_min:.2f}, max = {col_max:.2f}")

Min and Max of standardized features (X_train):
Bollinger_Upper: min = -0.56, max = 3.62
Bollinger_Lower: min = -0.55, max = 4.07
Fgi_Full: min = -1.41, max = 2.34
Close: min = -0.56, max = 4.05
nasdaq: min = -1.25, max = 2.70
sp500: min = -1.56, max = 2.66
vix: min = -1.20, max = 9.23
Volume: min = -0.24, max = 13.34


In [36]:



# Make predictions for the next closing price
# Use the last row from X_test (or any new data)
latest_data = X_test.iloc[-1:]  # Keep it as a DataFrame (not numpy array)

# Apply the same scaling to the latest data
latest_data_scaled = scaler.transform(latest_data)  # The feature names are kept

# Make prediction for the next closing price
next_close_pred = model.predict(latest_data_scaled)
print(f"Predicted next closing price: {next_close_pred[0]}")

Predicted next closing price: 75864.37813554882


In [37]:
# Extract the coefficients and the intercept
coefficients = model.coef_
intercept = model.intercept_

# Create a dictionary mapping feature names to their corresponding coefficients
feature_names = X.columns
equation = f"Predicted next closing price = {intercept:.2f}"

# Add each feature and its coefficient to the equation
for feature, coef in zip(feature_names, coefficients):
    equation += f" + ({coef:.2f}) * {feature}"

# Display the equation
print("Final equation for predicting the next closing price:")
print(equation)


Final equation for predicting the next closing price:
Predicted next closing price = 8177.87 + (367.85) * Bollinger_Upper + (-264.66) * Bollinger_Lower + (6.39) * Fgi_Full + (14451.99) * Close + (472.19) * nasdaq + (-358.75) * sp500 + (-16.11) * vix + (2.83) * Volume


In [38]:


import plotly.graph_objects as go

# Create a DataFrame to store the actual vs predicted values
results = pd.DataFrame({
    'Date': X_test.index,
    'Actual': y_test,
    'Predicted': y_pred
})

# Make sure Date is datetime (Plotly likes it)
results['Date'] = data.loc[X_test.index, 'Date'].values

# Build interactive plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Closing Price',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Predicted'],
    mode='lines',
    name='Predicted Closing Price',
    line=dict(color='red', dash='dash')
))

# Layout styling
fig.update_layout(
    title='Actual vs Predicted Closing Prices',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    legend=dict(x=0, y=1.1, orientation='h'),
    autosize=True,
    margin=dict(l=40, r=40, t=80, b=40),
    height=500,
)

# Show the plot
fig.show()



In [39]:
# Evaluate the model
r2 = r2_score(y_test, y_pred)

print(f"R-squared: {r2}")

mae = mean_absolute_error(y_test, y_pred)
# MSE (Mean Squared Error): average of the squared differences between predicted and actual values
# RMSE (Root Mean Squared Error): square root of MSE, gives error in original units
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

#how far off my predictions are from actual values
print(f"MAE: {mae:.2f}")

print(f"RMSE: {rmse:.2f}")

R-squared: 0.9968698896712773
MAE: 910.02
RMSE: 1449.81


In [40]:
# Calculate absolute error and rolling MAE
results['Absolute_Error'] = abs(results['Actual'] - results['Predicted'])
results['MAE_Rolling'] = results['Absolute_Error'].rolling(window=7).mean()
# Calculate squared error and rolling RMSE
results['Squared_Error'] = (results['Actual'] - results['Predicted']) ** 2
results['RMSE_Rolling'] = results['Squared_Error'].rolling(window=7).mean().pow(0.5)

# Plot Rolling MAE and RMSE on the same chart
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['RMSE_Rolling'],
    mode='lines',
    name='Rolling RMSE',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['MAE_Rolling'],
    mode='lines',
    name='Rolling MAE',
    line=dict(color='blue')
))

fig.update_layout(
    title='7-Day Rolling MAE and RMSE Over Time',
    xaxis_title='Date',
    yaxis_title='Error (USD)',
    template='plotly_white'
)

fig.show()
#fig.write_image("assets/Rolling_MAE_RMS.png")


In [41]:
import statsmodels.api as sm
import plotly.express as px

# Store original column names
original_columns = X_train.columns

# Standardize the features (X_train_scaled) for fitting the model
X_with_intercept = sm.add_constant(x_train_scaled)  # Add intercept term to the features
y_train_array = y_train.values  # Convert to numpy array for Statsmodels compatibility

# Fit the model using Statsmodels
model_sm = sm.OLS(y_train_array, X_with_intercept)  # Ordinary Least Squares Regression
results = model_sm.fit()  # Fit the model

# Display the summary, which includes the p-values for each feature
summary = results.summary()

# Map p-values to the original feature names
p_values = results.pvalues[1:]  # Exclude the intercept (constant)
features_with_p_values = dict(zip(original_columns, p_values))

# Print the p-values for each feature and check significance
print("Feature p-values (human-readable labels):")
for feature, p_value in features_with_p_values.items():
    significance = "Significant" if p_value < 0.05 else "Not Significant"
    print(f"{feature}: {p_value:.4f} ({significance})")

# Optionally, print the full summary
print("\nFull summary with p-values:")
print(summary)


# Convert to DataFrame for plotting
pval_df = pd.DataFrame({
    'Feature': list(features_with_p_values.keys()),
    'P_Value': list(features_with_p_values.values())
})

# Sort by P-value
pval_df = pval_df.sort_values('P_Value')

# Plot
fig = px.bar(
    pval_df,
    x='Feature',
    y='P_Value',
    color=pval_df['P_Value'] < 0.05,
    color_discrete_map={True: 'green', False: 'red'},
    title="Feature Significance (P-Values)",
    labels={'P_Value': 'P-Value', 'Feature': 'Feature'},
    template='plotly_white'
)

fig.update_layout(
    yaxis_range=[0, 1],
    showlegend=False
)

fig.add_hline(y=0.05, line_dash="dash", line_color="black")

fig.show()





Feature p-values (human-readable labels):
Bollinger_Upper: 0.0002 (Significant)
Bollinger_Lower: 0.0004 (Significant)
Fgi_Full: 0.5881 (Not Significant)
Close: 0.0000 (Significant)
nasdaq: 0.0000 (Significant)
sp500: 0.0002 (Significant)
vix: 0.1837 (Not Significant)
Volume: 0.7821 (Not Significant)

Full summary with p-values:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 2.644e+05
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        12:37:22   Log-Likelihood:                -33357.
No. Observations:                4221   AIC:                         6.673e+04
Df Residuals:                    4212   BIC:                         6.679e+04
Df Model:                           8 

In [42]:
import plotly.figure_factory as ff

# Select only numeric columns
numeric_df = data[features + ['Target']].copy()

# Calculate correlation matrix
corr_matrix = numeric_df.corr().round(2)

# Create heatmap
fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns.tolist(),
    y=corr_matrix.index.tolist(),
    colorscale='Blues',
    showscale=True,
    annotation_text=corr_matrix.values.round(2)
)

fig.update_layout(
    title='📈 Feature Correlation Heatmap (Pearson)',
    template='plotly_white',
    height=600
)

fig.show()

#fig.write_image("assets/Feature_Correlation.png")

In [43]:
import plotly.graph_objects as go

# Create a DataFrame to store the actual vs predicted values
results = pd.DataFrame({
    'Date': X_test.index,
    'Actual': y_test,
    'Predicted': y_pred
})

# Make sure Date is datetime (Plotly likes it)
results['Date'] = data.loc[X_test.index, 'Date'].values

# Build interactive plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Closing Price',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Predicted'],
    mode='lines',
    name='Predicted Closing Price',
    line=dict(color='red', dash='dash')
))

# Layout styling
fig.update_layout(
    title='Actual vs Predicted Closing Prices',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    legend=dict(x=0, y=1.1, orientation='h'),
    autosize=True,
    margin=dict(l=40, r=40, t=80, b=40),
    height=500,
)

# Show the plot
fig.show()


In [44]:
import plotly.express as px


# Assume results DataFrame contains 'Actual' and 'Predicted' columns

# Melt the DataFrame so we can compare both Actual and Predicted in a single boxplot
box_df = results.melt(
    value_vars=['Actual', 'Predicted'],
    var_name='Type',
    value_name='Price'
)

# Create boxplot
fig = px.box(
    box_df,
    x='Type',
    y='Price',
    color='Type',
    title='Distribution of Actual vs Predicted Prices',
    points='all',  # Show all data points (adds scatter to boxplot)
    template='plotly_white'
)

fig.update_layout(
    yaxis_title='BTC Price (USD)',
    xaxis_title='Type',
    showlegend=False
)

fig.show()
#fig.write_image("assets/ActualvsPredicted.png")

In [45]:
import plotly.graph_objects as go

# Calculate rolling metrics if not already done
results['Absolute_Error'] = abs(results['Actual'] - results['Predicted'])
results['MAE_Rolling'] = results['Absolute_Error'].rolling(window=7).mean()
results['Squared_Error'] = (results['Actual'] - results['Predicted']) ** 2
results['RMSE_Rolling'] = results['Squared_Error'].rolling(window=7).mean() ** 0.5

# Create figure with secondary y-axis
fig = go.Figure()

# Primary y-axis: Predicted price
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Predicted'],
    name='Predicted Price',
    mode='lines',
    line=dict(color='blue')
))

# Secondary y-axis: MAE
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['MAE_Rolling'],
    name='Rolling MAE',
    mode='lines',
    yaxis='y2',
    line=dict(color='orange', dash='dash')
))

# Secondary y-axis: RMSE
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['RMSE_Rolling'],
    name='Rolling RMSE',
    mode='lines',
    yaxis='y2',
    line=dict(color='red', dash='dot')
))

# Layout
fig.update_layout(
    title='Predicted BTC Price with Rolling MAE and RMSE',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Predicted Price (USD)', side='left'),
    yaxis2=dict(
        title='Error (USD)',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    legend=dict(x=0, y=1.1, orientation='h'),
    template='plotly_white',
    height=500
)

fig.show()


In [46]:
# Calculate % error metrics (avoid division by zero just in case)
results['MAE_PCT'] = (results['Absolute_Error'] / results['Predicted'].replace(0, 1)) * 100
results['RMSE_PCT'] = (results['RMSE_Rolling'] / results['Predicted'].replace(0, 1)) * 100

MAEWorstSpike = results['MAE_PCT'].max()
RMSEWorstSpike = results['RMSE_PCT'].max()

import plotly.graph_objects as go

fig = go.Figure()

fig.add_hline(
    y=3,
    line_dash="dash",
    line_color="black",
    annotation_text="3% Error Threshold",
    annotation_position="top left"
)

fig.add_hline(
    y=results['MAE_PCT'].mean(),
    line_dash="dash",
    line_color="purple",
    annotation_text="Average MAE",
    annotation_position="top left"
)

fig.add_hline(
    y=results['RMSE_PCT'].mean(),
    line_dash="dash",
    line_color="green",
    annotation_text="Average RMSE",
    annotation_position="top left"
)

fig.add_hline(
    y=MAEWorstSpike,
    line_dash="dash",
    line_color="green",
    annotation_text="Worst MAE Spike",
    annotation_position="top left"
)

fig.add_hline(
    y=RMSEWorstSpike,
    line_dash="dash",
    line_color="green",
    annotation_text="Worst RMSE Spike",
    annotation_position="top left"
)



fig.add_trace(go.Bar(
    x=results['Date'],
    y=results['MAE_PCT'],
    name='MAE %',
    marker_color='blue',
    opacity=0.6
))

fig.add_trace(go.Bar(
    x=results['Date'],
    y=results['RMSE_PCT'],
    name='RMSE %',
    marker_color='red',
    opacity=0.6
))

fig.update_layout(
    title='Prediction Error as % of Predicted BTC Price',
    xaxis_title='Date',
    yaxis_title='Error (%)',
    barmode='overlay',  # or 'group' if you want side-by-side bars
    template='plotly_white',
    height=500
)

fig.show()
#fig.write_image("assets/Prediction_Error_MAE_RMSE.png")



In [47]:
from scipy.stats import gaussian_kde

fig = go.Figure()


mae_vals = results['Absolute_Error'].dropna()
rmse_vals = results['Squared_Error'].dropna() ** 0.5

# Generate KDE for MAE
mae_kde = gaussian_kde(mae_vals)
mae_x = np.linspace(mae_vals.min(), mae_vals.max(), 200)
mae_y = mae_kde(mae_x)

# Generate KDE for RMSE
rmse_kde = gaussian_kde(rmse_vals)
rmse_x = np.linspace(rmse_vals.min(), rmse_vals.max(), 200)
rmse_y = rmse_kde(rmse_x)

# RMSE Histogram + KDE FIRST (goes behind)
fig.add_trace(go.Histogram(
    x=rmse_vals,
    name='RMSE',
    opacity=0.6,
    nbinsx=40,
    marker_color='red',
    xbins=dict(size=250)
))
fig.add_trace(go.Scatter(
    x=rmse_x,
    y=rmse_y * len(rmse_vals) * 250,
    name='RMSE KDE',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title='RMSE Error Distribution with KDE',
    xaxis_title='Error Value (USD)',
    yaxis_title='Frequency',
    template='plotly_white',
    legend=dict(x=0.75, y=0.95)
)


fig.show()
#fig.write_image("assets/RMSE_Error_Distribution.png")



In [49]:
# MAE Histogram + KDE only
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=mae_vals,
    name='MAE',
    opacity=0.6,
    nbinsx=40,
    marker_color='blue',
    xbins=dict(size=250)
))
fig.add_trace(go.Scatter(
    x=mae_x,
    y=mae_y * len(mae_vals) * 250,
    name='MAE KDE',
    line=dict(color='blue', width=2)
))

fig.update_layout(
    title='MAE Error Distribution with KDE',
    xaxis_title='Error Value (USD)',
    yaxis_title='Frequency',
    template='plotly_white',
    legend=dict(x=0.75, y=0.95)
)

fig.show()


In [50]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calculate residuals
results['Residual'] = results['Actual'] - results['Predicted']

# Create subplots: 2 rows, shared x-axis
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    row_heights=[0.7, 0.3],
    vertical_spacing=0.05,
    subplot_titles=("Distribution of Residuals", "Boxplot of Residuals")
)

# Histogram
fig.add_trace(
    go.Histogram(
        x=results['Residual'],
        nbinsx=50,
        name='Residuals',
        marker_color='lightskyblue',
        opacity=0.7
    ),
    row=1, col=1
)

# Boxplot
fig.add_trace(
    go.Box(
        x=results['Residual'],
        name='Residuals',
        marker_color='indianred',
        boxpoints='outliers'  # show only outliers
    ),
    row=2, col=1
)

fig.update_layout(
    height=600,
    template='plotly_white',
    showlegend=False,
    xaxis_title='Residual (Actual - Predicted)',
    yaxis_title='Count',
    title_text='📊 Residual Analysis: Histogram + Boxplot'
)

fig.show()


#fig.write_image("assets/Distribution_Residuals.png")




In [51]:
print("Actual price range:", results['Actual'].min(), "-", results['Actual'].max())
print("Predicted price range:", results['Predicted'].min(), "-", results['Predicted'].max())
print("Predicted price std dev:", results['Predicted'].std())


Actual price range: 15760.14 - 106159.26
Predicted price range: 15757.506888477228 - 105426.47239280111
Predicted price std dev: 25735.32800392257


In [52]:
import plotly.express as px

fig = px.scatter(
    results,
    x='Actual',
    y='Predicted',
    title='Actual vs Predicted Prices',
    labels={'Actual': 'Actual BTC Price', 'Predicted': 'Predicted BTC Price'},
    trendline='ols',
    template='plotly_white'
)

fig.show()
#fig.write_image("assets/ActualVsPredicted.png")


In [53]:
# Align predictions back to the original index
data.loc[results.index, 'Predicted'] = results['Predicted'].values

In [55]:
data.to_sql("BTC_ModelOutput", con=engine, if_exists="replace", index=False)


5277